# K-Means Clustering

The clustering method k-means aims to partition N observations into K clusters in which each observation belongs to the cluster with the nearest mean. The problem is NP hard. 

#### Mathematical background

The k-means algorithm takes a dataset X of N points as input, together with a parameter K specifying how many clusters to create. The output is a set of K cluster centroids and a labeling of X that assigns each of the points in X to a unique cluster. All points within a cluster are closer in distance to their centroid than they are to any other centroid.

The mathematical condition for the K clusters $C_k$ and the K centroids $\mu_k$ can be expressed as: 

Minimize $\displaystyle \sum_{k=1}^K \sum_{\mathrm{x}_n \in C_k} ||\mathrm{x}_n - \mu_k ||^2$ with respect to $\displaystyle C_k, \mu_k$

#### Lloyd's algorithm

An iterative method known as the k-means algorithm or Lloyd’s algorithm exists that converges (albeit to a local minimum) in few steps. The procedure alternates between two operations. 

1. Once a set of centroids $\mu_k$ is available, the clusters are updated to contain the points closest in distance to each centroid. 
2. Given a set of clusters, the centroids are recalculated as the means of all points belonging to a cluster.

$$\displaystyle C_k = \{\mathrm{x}_n : ||\mathrm{x}_n - \mu_k|| \leq \mathrm{\,\,all\,\,} ||\mathrm{x}_n - \mu_l||\}\qquad(1)$$

$$\displaystyle \mu_k = \frac{1}{C_k}\sum_{\mathrm{x}_n \in C_k}\mathrm{x}_n\qquad(2)$$

The two-step procedure continues until the assignments of clusters and centroids no longer change. As already mentioned, the convergence is guaranteed but the solution might be a local minimum. In practice, the algorithm is run multiple times and averaged. For the starting set of centroids, several methods can be employed, for instance random assignation.

The k-means algorithm works reasonably well when the data fits the cluster model:

* The clusters are spherical: the data points in a cluster are centered around that cluster
* The spread/variance of the clusters is similar: Each data point belongs to the closest cluster


In [ ]:
%matplotlib inline
import os, sys, random
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from shapely.geometry import Point, MultiPoint

In [ ]:
# Add path to JSAnimation to pythonpath 
# git clone https://github.com/jakevdp/JSAnimation.git
# Save JSAnimation in geodata/src
JSAnimation_path = os.path.join('./', '../src/JSAnimation')
sys.path.append(JSAnimation_path)
from JSAnimation import IPython_display

### A class implementation of the k-means algorithm

In [ ]:
# Class that implements the K-Means clustering algorithm
class KMeans():
    def __init__(self, K, N=None, init_data='random', X=None, verbose=True):
        
        # Init points -- in the domain (-1, 1)x(-1, 1)
        if init_data == 'random':
            try:
                self.X = [Point(random.uniform(-1, 1), random.uniform(-1, 1)) for i in range(N)]
            except TypeError:
                print('N needs to be provided if init_data=random')
                return
        elif init_data == 'file':
            # Override any given N, if any
            N = None
            self.X = []
            for line in open('X2.dat'):
                x, y = map(float, line.strip().split(','))
                self.X.append(Point(x, y))
        elif init_data == 'data':
            if not X:
                print('X needs to be specified when initializing with data')
                return
            # Override any given N, if any
            N = None
            self.X = X
        else:
            print('Method invalid')
            return
        
        # Set attributes
        self.K = K
        self.N = N or len(self.X)
        self.MAX_ITERATIONS = 100
        self.verbose = verbose

        # Sanity check for input parameters
        if self.N <= 1:
            print('Try N > 1 for a meaningful problem.')
            return
        if self.K > self.N:
            print('The number of target clusters is larger than the number of available points. Try K <= N.')
            return
        
        print('Initialized problem with {} points to be clustered to {}'.format(self.N, self.K)) if self.verbose else ''
        
    # Function to initialize centroids randomly among the data points
    def _init_centroids(self):
        print('Initialized centroids assigning to random dataset points.') if self.verbose else ''
        return {label: centroid for label, centroid in enumerate(random.sample(self.X, self.K))}
        
    # Function to assign points to each centroid
    def _cluster_points(self):
        # ASSIGNMENT
        # Implement method to assign all points self.X to self.clusters 
        # according to the closest centroid (in self.centroids)
        raise NotImplementedError
        
    def _reassign_centroids(self):
        # Create a MultiPoint with all points in each cluster. New centroid can be trivially computed
        centroids = {}
        # Method .items() of dictionary returns an iterator on (key, values)
        for label, cluster in self.clusters.items():
            centroids[label] = self.mp(cluster).centroid
        self.centroids = centroids
        
    def _has_converged(self):
        # Decides whether the algorithm has reached an end point
        # Conditions to exit: 
        # (1) reach max number of allowed iterations
        # (2) algorithm has converged and found a stable solution
        if self.iteration > self.MAX_ITERATIONS: 
            return False
        # In the first iteration, old_centroids has not been set yet
        # Except with AttributeError (EAFP way: easier to ask for forgiveness than permission)
        try:
            return set(tuple(c.coords) for c in self.centroids.values()) == \
                    set(tuple(c.coords) for c in self.old_centroids.values())
        except AttributeError:
            return False

    def initialize(self, plot=False):
        
        # Initialize attributes to empty state (allows to rerun the .run() method on the same initial data)
        self.iteration = 0
        self.centroids = None
        self.clusters = None
        self.data = []
        
        # Store initial data (for later viz). Note that there're no centroids in this 1st step
        this_data = {0: {'cluster': self.X}}
        self.data.append(this_data)
        
        # Initialize centroids
        self.centroids = self._init_centroids()
        
        # To visualize initialization of centroids
        if plot:
            fig = plt.figure(figsize=(4, 4))
            ax = fig.add_axes([0, 0, 1, 1], xlim=(-1, 1), ylim=(-1, 1), aspect='equal', xticks=[], yticks=[])
            
            # Plot data points
            ax.plot([p.x for p in self.X], [p.y for p in self.X], 'ro', markersize=4, alpha=0.3)
            
            # Plot centroids, annotate them according to the order in which they were drawn
            # Sorted list by kmeans.centroid keys
            centroids = [c[1] for c in sorted(list(self.centroids.items()))]
            ax.plot([c.x for c in centroids], [c.y for c in centroids], 'bo', markersize=6, alpha=1)
            for i, c in enumerate(centroids, 1):
                ax.text(c.x, c.y, str(i), fontsize=20)
                
    def run(self):
        
        # Loop until algorithm converges
        while not self._has_converged():
            # Update counter
            self.iteration += 1

            # Copy existing centroids to old_centroids
            self.old_centroids = self.centroids
            
            # Assign all points in X to clusters
            self._cluster_points()
            
            # Store data for plotting. Each cluster has an integer label and two components: centroid and cluster
            # centroid is a point and cluster a list of points
            this_data = {}
            for label in self.centroids:
                this_data[label] = {'centroid': self.centroids[label], 'cluster': self.clusters[label]}
            self.data.append(this_data)
            
            # Reevaluate centroids according to the new clusters
            self._reassign_centroids()
            
        # Exit
        print('Clustered {} points to {} clusters in {} iterations. Bye'.
              format(self.N, self.K, self.iteration-1)) if self.verbose else ''
    
    def animator(self):
        
        # Utility function to create an animated video of the evolution
        # Initialize figure and axes. 
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_axes([0, 0, 1, 1], xlim=(-1, 1), ylim=(-1, 1), aspect='equal', xticks=[], yticks=[])
        ax.spines['bottom'].set_linewidth(3)
        ax.spines['right'].set_linewidth(3)
        
        # Create a list of K colors taken from a particular colormap
        # See http://wiki.scipy.org/Cookbook/Matplotlib/Show_colormaps
        colors = plt.cm.spectral([1.*m/(self.K-1) for m in range(self.K)])
        
        # stars is the list of K layers for the centroids, pnts is for the clusters
        # Initialize with empty 2d data and the desired marker, size and transparency
        stars = sum([ax.plot([], [], '*', markersize=10, alpha=1) for k in range(self.K)], [])
        pnts = sum([ax.plot([], [], '.', markersize=8, alpha=0.6) for k in range(self.K)], [])
        
        def _init():
            # The initial data are all the unclustered points
            data_snapshot = self.data[0]
            # Set color to red and add the only cluster (all points) to the layer
            pnts[0].set_color('red')
            cluster = data_snapshot[0]['cluster']
            pnts[0].set_data([c.x for c in cluster], [c.y for c in cluster])
            return pnts

        def _animate(i):
            # On first iteration don't do anything, that's the initial data
            if i == 0:
                return
            # For each subsequent iteration, get the snapshot of the data
            data_snapshot = self.data[i]
            
            # Populate stars with centroid data and pnts with cluster points
            for k, (star, pnt) in enumerate(zip(stars, pnts)):
                # Choose always the same color for the same label!
                c = colors[k]
                centroid = data_snapshot[k]['centroid']
                cluster = data_snapshot[k]['cluster']
                # Set color to the markers
                star.set_color(c)
                pnt.set_color(c)
                # Centroid is a single point
                star.set_data(centroid.x, centroid.y)
                # Cluster is a list of points. .plot() takes arrays of x and y positions
                pnt.set_data([c.x for c in cluster], [c.y for c in cluster])
            return stars + pnts
        
        # Create an animation of as many frames as iterations, interval in ms
        # blit=True only re-draws parts of plot that change
        anim = animation.FuncAnimation(fig, _animate, init_func=_init,
                                       frames=self.iteration, interval=600, blit=False)
        return anim
        
    # Turns a list of geometric point features into a MultiPoint (for easy viz)
    def mp(self, list_of_points):
        return MultiPoint([p for p in list_of_points])

### Running k-means on random data with random initialization of centroids

In a random data sample there is no obvious cluster structure. The solution is alike to solving for a continuous geometric region, giving raise to a Voronoi tessellation of space. 

In [ ]:
kmeans = KMeans(K=5, N=500)
kmeans.initialize(plot=True)

In [ ]:
kmeans.run()

In [ ]:
anim = kmeans.animator()
IPython_display.display_animation(anim, default_mode='once')

Due to the random initialization of centroids, each instantiation of the algorithm leads to a different solution. Potentially, only local minima are reached. 

### Exercises

1. Compare a second run over the same data and spot the differences in final configuration.
2. Rerun the clustering algorithm with a different number of K centroids on N=500 points
3. Observe the Voronoi tessellation by running with a large number of points N>500 (careful about performance!)

### Running k-means on globular data with random initialization of centroids

In a data sample with defined cluster-like structures, the k-means algorithm initialized to random centroids may produce a suboptimal solution if the initial random draw is unfortunate, which is an obvious drawback of the initialization method.

In [ ]:
kmeans3 = KMeans(K=3, init_data='file')
kmeans3.initialize(plot=True)
kmeans3.run()

In [ ]:
anim3 = kmeans3.animator()
IPython_display.display_animation(anim3, default_mode='once')

### Exercises

1. Compare with a different evolution on the same data.
2. Run the algorithm a statistically significant number of times and annotate the percentage of times that the algorithm converges to the 'right' solution.
3. Why is the algorithm converging to a suboptimal solution? Think of possible ways of improving the initialisation to avoid falling into a suboptimal solution.

### k-means++: the advantages of careful seeding

A solution to the inadequacy of the random initialization of centroid, called k-means++, was proposed in 2007 by [Arthur and Vassilvitskii](http://theory.stanford.edu/~sergei/papers/kMeansPP-soda.pdf). This algorithm comes with a theoretical guarantee to find a solution that is $O(log k)$ competitive to the optimal k-means solution. It is also fairly simple to describe and implement. Starting with a dataset X of N points $(\mathrm{x}_1, \ldots, \mathrm{x}_N)$,

* choose an initial center $c_1$ uniformly at random from X. Compute the vector containing the square distances between all points in the dataset and $c_1: D_i^2 = ||\mathrm{x}_i - c_1 ||^2$
* choose a second center $c_2$ from X randomly drawn from the probability distribution $D_i^2 / \sum_j D_j^2$
* recompute the distance vector as $D_i^2 = \mathrm{min} \left(||\mathrm{x}_i - c_1 ||^2, ||\mathrm{x}_i - c_2 ||^2\right)$
* choose a successive center $c_l$ and recompute the distance vector as $D_i^2 = \mathrm{min} \left(||\mathrm{x}_i - c_1 ||^2, \ldots, ||\mathrm{x}_i - c_l ||^2\right)$
* when exactly k centers have been chosen, finalize the initialization phase and proceed with the standard k-means algorithm

To implement the a probability distribution of step 2, we compute the cumulative probabilities for choosing each of the N points in X. These cumulative probabilities are partitions in the interval [0,1] with length equal to the probability of the corresponding point being chosen as a center, as explained in this stackoverflow thread. Therefore, by picking a random value $r \in [0,1]$ and finding the point corresponding to the segment of the partition where that $r$ value falls, we are effectively choosing a point drawn according to the desired probability distribution.

In [ ]:
class KPlusPlus(KMeans):
    
    def _dist_from_centroids(self, centroids):
    
        # Compute distance vector: 
        # for each point in X, find the minimum of the square distances to each of the available centroids
        D2 = np.array([min([x.distance(c)**2 for c in centroids.values()]) for x in self.X])
        self.D2 = D2
 
    def _choose_next_centroid(self):
        
        # Construct the probability distribution D2_i/sum(D2_j)
        probs = self.D2/self.D2.sum()
        # Compute the cumulative distribution
        cumprobs = probs.cumsum()
        # Find in which partition of the [0, 1] interval according to the cum distribution a random number falls
        r = random.random()
        ind = np.where(cumprobs >= r)[0][0]
        return(self.X[ind])
 
    def _init_centroids(self):
        
        # Draw first centroid randomly
        centroids = {0: random.sample(self.X, 1)[0]}
        drawn_centroids = len(centroids.values())
        # Continue drawing until having K centroids
        while drawn_centroids <= self.K:
            self._dist_from_centroids(centroids)
            centroids[drawn_centroids-1] = self._choose_next_centroid()
            drawn_centroids += 1
            
        print('Initialized centroids assignment using k++') if self.verbose else ''

        return centroids

In [ ]:
kpp = KPlusPlus(K=3, init_data='file')
kpp.initialize(plot=True)
kpp.run()

In [ ]:
animpp = kpp.animator()
IPython_display.display_animation(animpp, default_mode='once')

### Exercise

1. Run the k++ algorithm a statistically significant number of times and annotate the percentage of times that the algorithm converges to the 'right' solution. Compare with the result that you found when running the naive initialisation above.

### Finding the K in k-means clustering

Clustering consist of grouping objects in sets, such that objects within a cluster are as similar as possible, whereas objects from different clusters are as dissimilar as possible. Thus, the optimal clustering is somehow subjective and dependent on the characteristic used for determining similarities, as well as on the level of detail required from the partitions. 

A particular disadvantage of the kmeans algorithm is the need for an a priori specification of the target number of clusters K. Is it possible to determine this number from the data per se?

An example: is the data from file composed of two or three clusters?

In [ ]:
kpp2 = KPlusPlus(K=2, init_data='file')
kpp2.initialize()
kpp2.run()
animpp2 = kpp2.animator()
IPython_display.display_animation(animpp2, default_mode='once')

A promising approach to determine K is presented in a publication by [Pham, Dimov and Nguyen](http://www.ee.columbia.edu/~dpwe/papers/PhamDN05-kmeans.pdf). The article is very much worth reading, as it includes an explanation of the drawbacks of the standard k-means algorithm as well as a comprehensive survey on different methods that have been proposed for selecting an optimal number of clusters.

The goal of the authors is to define a function $f(K)$ to evaluate the quality of the clustering for different values of K. 

The distorsion of a cluster is a measure of the distance between points in a cluster and its centroid:

$$\displaystyle I_j = \sum_{\mathrm{x}_i \in C_j} ||\mathrm{x}_i - \mu_j ||^2.$$

The global impact of all clusters’ distortions is given by the quantity

$$\displaystyle S_k = \sum_{j=1}^K I_j.$$

The authors finally arrive at the following definition of $f(K)$.


In [ ]:
from IPython.display import Image
Image(filename='fk.png', width=400) 

$N_d$ is the number of dimensions (attributes) of the data set (2 in our case of 2-d points) and $\alpha_K$ is a weight factor. With this definition, $f(K)$ is the ratio of the real distortion to the estimated distortion and decreases when there are areas of concentration in the data distribution. Values of K that yield small $f(K)$ can be regarded as giving well-deﬁned clusters.

In [ ]:
class KFinder():
    def __init__(self, init_data='data', kmax=6, N=None, X=None, verbose=False):
        rangeK = range(1, kmax+1)
        kruns = []
        if init_data == 'data':
            # To do, exercise
            pass
        elif init_data in ['random', 'file']:
            for thisk in rangeK:
                kruns.append(KPlusPlus(K=thisk, N=N, init_data=init_data, verbose=verbose))
        
        self.rangeK = rangeK
        self.kruns = kruns
        self.verbose = verbose
        
    def _fK(self, krun, thisk, Skm1=0):
        # Compute f(K)
        # Nd is hardcoded to 2 in our case with 2d points
        Nd = 2
        # Recursive definition of alpha
        a = lambda k, Nd: 1 - 3/(4*Nd) if k == 2 else a(k-1, Nd) + (1-a(k-1, Nd))/6

        centroids = krun.centroids.values()
        clusters = krun.clusters.values()

        # Compute Sk quantity as sum of all clusters' distorsions
        Sk = sum([sum([p.distance(krun.centroids[i]) for p in krun.clusters[i]]) for i in range(thisk)])

        if thisk == 1:
            fs = 1
        elif Skm1 == 0:
            fs = 1
        else:
            fs = Sk/(a(thisk,Nd)*Skm1)
        return fs, Sk 
    
    def run(self):
        # We will store results as tuple of tuples [k, f(K)]
        fks = []
        
        # Run k++ for each of the target Ks
        for thisk, krun in zip(self.rangeK, self.kruns):
            krun.initialize(plot=self.verbose)
            krun.run()
            
            # Handle special k=1 case
            print('Computing fK for k={} using krun with k={}'.format(thisk, krun.K)) if self.verbose else ''
            if thisk == 1:
                fk, Sk = self._fK(krun, thisk)
            else:
                # Note that the value stored in variable Sk when thisk = K corresponds to SK-1
                # Therefore we are passing SK-1 as argument for the f(K) calculation and updating Sk in the same step
                fk, Sk = self._fK(krun, thisk, Sk)
            
            # Store results
            fks.append([thisk, fk])
            
        self.fks = fks

In [ ]:
kfinder = KFinder(init_data='file', kmax=8, verbose=True)

In [ ]:
kfinder.run()

In [ ]:
kfinder.fks

The f(K) paper suggests that values of f(K) < 0.85 are evidence of a potentially good value of K. 

In the case of our data we find a best value K=2, followed by a second best choice K=3.

In [ ]:
fig = plt.figure(figsize=(6, 4))
ax = fig.add_axes([0, 0, 1, 1])
ax.plot([k[0] for k in kfinder.fks], [k[1] for k in kfinder.fks], 'ro-', markersize=4, alpha=0.8)
ax.set_xlabel('Number of clusters K', fontsize=14)
ax.set_ylabel('Function f(K)', fontsize=14)
plt.show()

#### Exercise

Repeat this analysis using initial data with diverse cluster structures and find out which optimal K is found by the f(K) method.